This notebook contains code for implementing the Heston model and calibrating model parameters, followed by Monte Carlo simulations to validate the model.

# Importing Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
import plotly.graph_objects as go

from scipy.integrate import quad
from scipy.optimize import minimize
from datetime import datetime as dt

from nelson_siegel_svensson.calibrate import calibrate_nss_ols

# Implementing Heston Model
Heston model known parameters (from market data):
- initial asset price (S0) 
- risk-free interest rate (r) 
- time to maturity(T) 
- strike price (K)

unknown parameters (determined through optimization algorithms and calibration techniques):
- initial volatility (v0),
- speed of mean-reversion (kappa)
- long term mean of volatility (theta)
- volatility of volatility (sigma)
- correlation between the two wiener processes for asset price and volatility (rho)

Heston equations (SDEs, PDE, characteristic function) are from this paper:
https://www.maths.univ-evry.fr/pages_perso/crepey/Finance/051111_mikh%20heston.pdf

In [2]:
# Defining heston model characteristic function
# Characteristic function derived from performing fourier transform on its probability distribution then solving form

def heston_char_func(phi, S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd):

    # commonly used term
    rspi = rho*sigma*phi*1j

    # constants
    a = kappa*theta
    b = kappa+lambd

    # d and g parameter in heston characteristic function
    d = np.sqrt((rspi - b)**2 + sigma**2 * (phi*1j+phi**2))
    g = (b-rspi+d) / (b-rspi-d)

    exp1 = np.exp(r*phi*1j*tau)
    term1 = S0**(1j*phi) * ((1-g*np.exp(d*tau))/(1-g))**(-2*a/sigma**2)
    exp2 = np.exp(a*tau/sigma**2 * (b-rspi+d) + v0/sigma**2 * (b-rspi+d)*((1-np.exp(d*tau))/(1-g*np.exp(d*tau))))

    return exp1*term1*exp2

In [3]:
# Using numerical integration to obtain closed form for call price
def heston_call_price(S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd):
    args = (S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd)
    
    # 10000 steps and range of 1-100, each step (dphi) is 0.01
    P, umax, N = 0, 100, 10000
    dphi=umax/N 
    
    # Loop through all the steps and summing the value of integral at each step
    for i in range (1,N):
        phi = dphi * (2*i + 1)/2
        
        # P is 0 intially, and the value of the integral * dphi is added to P each step 
        P += ((np.exp(r*tau)*heston_char_func(phi-1j,*args) - K * heston_char_func(phi,*args)) / (1j*phi*K**(1j*phi))) * dphi

    # Substituting the value of the integral into equation for cost and taking the real value
    return np.real((S0 - K*np.exp(-r*tau))/2 + P/np.pi)

In [4]:
S0 = 105
K = 110
v0 = 0.04
tau = 0.1
r = 0.05
sigma = 0.6
rho = -0.8
kappa = 3
theta = 0.04
lambd = 0.8

heston_call_price(S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd)

np.float64(0.5687456253728729)

# Getting Real World Data
## Risk Free Interest Rate

In [5]:
rates_df = pd.read_csv('/Users/wongmarco/Downloads/daily-treasury-rates.csv')
rates_df['Date'] = pd.to_datetime(rates_df['Date'], format='%m/%d/%Y')
rates_df

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2023-12-29,5.60,5.59,5.40,5.41,5.26,4.79,4.23,4.01,3.84,3.88,3.88,4.20,4.03
1,2023-12-28,5.57,5.55,5.45,5.42,5.28,4.82,4.26,4.02,3.83,3.84,3.84,4.14,3.98
2,2023-12-27,5.55,5.53,5.44,5.42,5.26,4.79,4.20,3.97,3.78,3.81,3.79,4.10,3.95
3,2023-12-26,5.53,5.52,5.45,5.44,5.28,4.83,4.26,4.05,3.89,3.91,3.89,4.20,4.04
4,2023-12-22,5.54,5.52,5.44,5.45,5.31,4.82,4.31,4.04,3.87,3.92,3.90,4.21,4.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2023-01-09,4.37,4.58,4.70,4.74,4.83,4.69,4.19,3.93,3.66,3.60,3.53,3.83,3.66
246,2023-01-06,4.32,4.55,4.67,4.74,4.79,4.71,4.24,3.96,3.69,3.63,3.55,3.84,3.67
247,2023-01-05,4.30,4.55,4.66,4.75,4.81,4.78,4.45,4.18,3.90,3.82,3.71,3.96,3.78
248,2023-01-04,4.20,4.42,4.55,4.69,4.77,4.71,4.36,4.11,3.85,3.79,3.69,3.97,3.81


In [6]:
# Unused code: we are simplifying it by using only the 1 Month par yield curve rates to look at options with short maturitities
# This can be used in the future for a range of maturities and for better accuracy (curving it so it is more accurate daily)

# Using Nelson Siegel Svennson model (parametric) to analyse yield curve using ordinary least squares
# maturities = rates_df.columns.astype(float).to_numpy()
# rates_df['curve_fit'] = rates_df.apply(lambda row: calibrate_nss_ols(maturities, row.values)[0], axis=1)

## Option data
Option data is downloaded from optionsdx in form of CSV files (This project doesn't require real time quoting of option data)

In [7]:
# File path below contains EOD option data for NVDA in 2023 (we will be using 2023 Jan to Nov data to estimate option prices for Dec 2023 )
file_path = ['/Users/wongmarco/Downloads/nvda_eod_2023q1-0x56e5/nvda_eod_202301.txt','/Users/wongmarco/Downloads/nvda_eod_2023q1-0x56e5/nvda_eod_202302.txt','/Users/wongmarco/Downloads/nvda_eod_2023q1-0x56e5/nvda_eod_202303.txt','/Users/wongmarco/Downloads/nvda_eod_2023q2-miyyse/nvda_eod_202304.txt','/Users/wongmarco/Downloads/nvda_eod_2023q2-miyyse/nvda_eod_202305.txt','/Users/wongmarco/Downloads/nvda_eod_2023q2-miyyse/nvda_eod_202306.txt','/Users/wongmarco/Downloads/nvda_eod_2023q3-vzquiq/nvda_eod_202307.txt','/Users/wongmarco/Downloads/nvda_eod_2023q3-vzquiq/nvda_eod_202308.txt','/Users/wongmarco/Downloads/nvda_eod_2023q3-vzquiq/nvda_eod_202309.txt','/Users/wongmarco/Downloads/nvda_eod_2023q4-jf5cdq/nvda_eod_202310.txt','/Users/wongmarco/Downloads/nvda_eod_2023q4-jf5cdq/nvda_eod_202311.txt']


dataframes = []
for file in file_path:
    df = pd.read_csv(file)  # Use pd.read_excel() for Excel files, etc.
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
options_df = pd.concat(dataframes, ignore_index=True)

/var/folders/ts/wphp397x1197fdj1s_5wjk040000gn/T/ipykernel_12599/1007104248.py:7: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)  # Use pd.read_excel() for Excel files, etc.
/var/folders/ts/wphp397x1197fdj1s_5wjk040000gn/T/ipykernel_12599/1007104248.py:7: DtypeWarning: Columns (8,9,10,11,12,15,17,18,20,21,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)  # Use pd.read_excel() for Excel files, etc.


Data Cleaning and Transformation

In [8]:
pd.set_option('display.max_columns', 33)
options_df.columns = options_df.columns.str.replace('[\[\] ]', '', regex=True)
options_df['QUOTE_DATE'] = pd.to_datetime(options_df['QUOTE_DATE'])
options_df['EXPIRE_DATE'] = pd.to_datetime(options_df['EXPIRE_DATE'])
options_df = options_df.replace(' ',np.nan )
options_df[['C_VOLUME','C_ASK','C_BID']] = options_df[['C_VOLUME','C_ASK','C_BID']].astype(float)
options_df.dropna()
# Removing inaccurate data where days to expiry is 0
options_df = options_df[options_df['DTE'] != 0]

# Removing inaccurate data where option last trading price = 0 and trade volume is null 
options_df = options_df[(options_df['C_LAST'] != 0) & (options_df['P_LAST'] != 0)]


In [9]:
options_df
# Columns in dataframe: 
    # Time: (time in unix, quote time, quote date, quote hour of time, expire date, expire time in unix, day to expiration)
    # Greeks: Delta (Call and Put), Gamma, Vega, Theta, Rho
    # Other data (Call and Put):  implied volatility, trading volume, last traded price, size (open interest)
    # Strike: Strike price, strike distance (absolute/ percentage distance) between stock and strike price

,QUOTE_UNIXTIME,QUOTE_READTIME,QUOTE_DATE,QUOTE_TIME_HOURS,UNDERLYING_LAST,EXPIRE_DATE,EXPIRE_UNIX,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,C_SIZE,C_BID,C_ASK,STRIKE,P_BID,P_ASK,P_SIZE,P_LAST,P_DELTA,P_GAMMA,P_VEGA,P_THETA,P_RHO,P_IV,P_VOLUME,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT
0,1672779600,2023-01-03 16:00,2023-01-03,16.0,143.15,2023-01-06,1673038800,3.0,0.99562,0.00029,0.00165,-0.06241,0.00696,3.174680,2.0,76.9,1 x 1,78.15,78.35,65.0,0.0,0.01,0 x 43,0.02,-0.00056,0.0001,0.00035,-0.00402,0.0,2.468900,0.000000,78.2,0.546
1,1672779600,2023-01-03 16:00,2023-01-03,16.0,143.15,2023-01-06,1673038800,3.0,0.99658,0.00025,0.00138,-0.04172,0.00732,2.732770,0.0,92.25,1 x 1,73.15,73.30,70.0,0.0,0.01,0 x 43,0.04,-0.0005,0.00009,0.00026,-0.0046,0.0,2.247870,0.000000,73.2,0.511
2,1672779600,2023-01-03 16:00,2023-01-03,16.0,143.15,2023-01-06,1673038800,3.0,0.99863,0.00013,0.00059,-0.02118,0.00808,2.229220,0.0,69.26,1 x 1,68.10,68.30,75.0,0.0,0.01,0 x 43,0.01,-0.00028,0.00004,-0.00002,-0.00404,-0.00037,2.042420,0.000000,68.2,0.476
3,1672779600,2023-01-03 16:00,2023-01-03,16.0,143.15,2023-01-06,1673038800,3.0,1.0,0.0,0.0,-0.0102,0.00838,NaN,0.0,64.21,35 x 35,61.20,64.40,80.0,0.0,0.01,0 x 43,0.01,-0.00107,0.00008,-0.00011,-0.00485,-0.00043,1.849050,0.000000,63.2,0.441
4,1672779600,2023-01-03 16:00,2023-01-03,16.0,143.15,2023-01-06,1673038800,3.0,1.0,0.0,0.0,-0.01147,0.00835,0.000050,0.0,55.5,37 x 35,57.15,59.20,85.0,0.0,0.01,0 x 31,0.02,-0.00025,0.00016,0.00074,-0.00465,0.0,1.667000,0.000000,58.2,0.406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468230,1701378000,2023-11-30 16:00,2023-11-30,16.0,467.70,2026-01-16,1768597200,778.0,0.29558,0.00123,2.32871,-0.07258,2.09956,0.411170,0.0,34.5,70 x 92,39.25,40.20,840.0,368.75,374.85,47 x 35,359.25,-1.0,0.0,0.0,0.0,0.0,NaN,0.000000,372.3,0.796
468231,1701378000,2023-11-30 16:00,2023-11-30,16.0,467.70,2026-01-16,1768597200,778.0,0.2874,0.00115,2.30113,-0.07138,2.05323,0.410120,0.0,55.04,64 x 104,37.75,38.95,850.0,377.9,385.9,32 x 5,373.0,-1.0,0.0,0.0,0.0,0.0,NaN,0.000000,382.3,0.817
468232,1701378000,2023-11-30 16:00,2023-11-30,16.0,467.70,2026-01-16,1768597200,778.0,0.28099,0.00119,2.27554,-0.07032,2.01066,0.409860,0.0,55.8,88 x 95,36.65,37.75,860.0,387.4,395.35,32 x 32,379.42,-1.0,0.0,0.0,0.0,0.0,NaN,0.000000,392.3,0.839
468234,1701378000,2023-11-30 16:00,2023-11-30,16.0,467.70,2026-01-16,1768597200,778.0,0.26792,0.00114,2.22572,-0.06884,1.92965,0.410230,0.0,43.9,49 x 105,34.70,35.40,880.0,404.0,418.85,25 x 25,428.81,-1.0,0.0,0.0,0.0,0.0,NaN,0.000000,412.3,0.882


In [125]:
# Data processing

# Getting a dataframe with only the parameters needed for the actively traded call options (for short maturities)
traded_options = options_df[(options_df['C_VOLUME'] > 500)&(options_df['DTE']>7)&(options_df['DTE']<366)][['UNDERLYING_LAST','QUOTE_DATE','DTE','C_VOLUME','C_LAST','C_SIZE','C_BID','C_ASK','STRIKE']]

# Change DTE from days to year, and using C_BID and C_ASK to estimate market price (Bid-Ask spread)
traded_options['DTE'] = traded_options['DTE']/365
traded_options = traded_options.rename(columns={'DTE': 'MATURITIES'})
traded_options['MARKET_PRICE'] = (traded_options['C_ASK']+traded_options['C_BID'])/2

# Taking away the very in or out of money options for better volatility surface
traded_options = traded_options[(traded_options['MARKET_PRICE']<100)&(traded_options['MARKET_PRICE']>1)]

# Risk free interest rate estimation (using 1 month as we will be examining short term options) and interpolating for missing data
traded_options = pd.merge(traded_options, rates_df[['Date','1 Mo']], left_on = 'QUOTE_DATE', right_on = 'Date', how='outer')
traded_options = traded_options.rename(columns={'1 Mo': 'RISK_FREE_RATE'})
traded_options['RISK_FREE_RATE'] = traded_options['RISK_FREE_RATE']/100
traded_options['RISK_FREE_RATE'] = traded_options['RISK_FREE_RATE'].interpolate(method = 'nearest')

# Dropping unused columns and nulls
traded_options = traded_options.dropna()

# Only using 6 months of data since this stock is volatile
traded_options = traded_options[traded_options['Date'].dt.month.isin(range(11, 12))]

traded_options


,UNDERLYING_LAST,QUOTE_DATE,MATURITIES,C_VOLUME,C_LAST,C_SIZE,C_BID,C_ASK,STRIKE,MARKET_PRICE,Date,RISK_FREE_RATE
4245,423.11,2023-11-01,0.024767,683.0,23.15,7 x 1,22.55,23.35,405.0,22.950,2023-11-01,0.0556
4246,423.11,2023-11-01,0.024767,510.0,16.0,24 x 2,15.25,15.80,415.0,15.525,2023-11-01,0.0556
4247,423.11,2023-11-01,0.024767,2024.0,12.95,4 x 6,12.35,12.75,420.0,12.550,2023-11-01,0.0556
4248,423.11,2023-11-01,0.024767,629.0,10.15,7 x 10,10.10,10.20,425.0,10.150,2023-11-01,0.0556
4249,423.11,2023-11-01,0.024767,1178.0,7.95,18 x 8,7.80,7.80,430.0,7.800,2023-11-01,0.0556
...,...,...,...,...,...,...,...,...,...,...,...,...
4543,467.70,2023-11-30,0.136986,1561.0,12.15,46 x 181,11.95,12.10,500.0,12.025,2023-11-30,0.0556
4544,467.70,2023-11-30,0.136986,800.0,9.4,13 x 177,9.25,9.40,510.0,9.325,2023-11-30,0.0556
4545,467.70,2023-11-30,0.136986,1186.0,3.95,111 x 202,4.00,4.15,540.0,4.075,2023-11-30,0.0556
4546,467.70,2023-11-30,0.136986,1327.0,3.1,305 x 75,3.00,3.10,550.0,3.050,2023-11-30,0.0556


In [11]:
# unused code for applying curve to interest rate
"""
# Mapping Nelson Siegel Svensson result function to date, then applying it to the list of maturities of the same date
rates_df = rates_df.reset_index()
function_mapping = dict(zip(rates_df['Date'], rates_df['curve_fit']))
traded_options = pd.merge(traded_options, rates_df[['Date','curve_fit']], left_on = 'QUOTE_DATE', right_on = 'Date', how='outer')

def apply_function(row):
    func = function_mapping.get(row['Date'])
    if func:
        return func(row['MATURITIES'])
    else:
        return np.nan 

traded_options['RISK_NEUTRAL_RATE'] = traded_options.apply(apply_function, axis=1)

# Interpolation to fill missing risk neutral rates
traded_options['RISK_NEUTRAL_RATE'] = traded_options['RISK_NEUTRAL_RATE'].interpolate(method = 'nearest')
traded_options = traded_options.drop(columns=['Date', 'curve_fit'])"""


"\n# Mapping Nelson Siegel Svensson result function to date, then applying it to the list of maturities of the same date\nrates_df = rates_df.reset_index()\nfunction_mapping = dict(zip(rates_df['Date'], rates_df['curve_fit']))\ntraded_options = pd.merge(traded_options, rates_df[['Date','curve_fit']], left_on = 'QUOTE_DATE', right_on = 'Date', how='outer')\n\ndef apply_function(row):\n    func = function_mapping.get(row['Date'])\n    if func:\n        return func(row['MATURITIES'])\n    else:\n        return np.nan \n\ntraded_options['RISK_NEUTRAL_RATE'] = traded_options.apply(apply_function, axis=1)\n\n# Interpolation to fill missing risk neutral rates\ntraded_options['RISK_NEUTRAL_RATE'] = traded_options['RISK_NEUTRAL_RATE'].interpolate(method = 'nearest')\ntraded_options = traded_options.drop(columns=['Date', 'curve_fit'])"

# Calibration of data using a least squared error fit
To find the set of parameters that minimizes the square error:
sqErr(v0, kappa, theta, sigma, rho, lambd) = sum of (Market call price - heston model call price) squared 

Using Scipy (optimization) for minimizing sqErr:
- Problem 1: selecting suitable weight terms (for better calibration results: fitting to more important/reliable data) and penalty  (to avoid overfitting)

- Problem 2: given the problem (minimizing square err), what is the most suitable optimization method to use
    - non linear problem with non linear constraints -> 

- Problem 3: selecting suitable initial parameters and bounds (for convergence speed, avoiding local minima, good quality result)

The result for the parameters should be at a reasonable value, and have a low error when comparing model and real price



In [165]:
# Variables from options data
S0, K, r, tau, price = traded_options[['UNDERLYING_LAST', 'STRIKE', 'RISK_FREE_RATE', 'MATURITIES','MARKET_PRICE']].astype(float).to_numpy().T

# Parameters （Setting initial guess and upper lower bound)
params = {
    "v0": {"x0": 0.15, "lbub": [1e-3, 0.4]},
    "kappa": {"x0": 2, "lbub": [1e-2, 5]},
    "theta": {"x0": 0.1, "lbub": [1e-3, 0.3]},
    "sigma": {"x0": 0.5, "lbub": [1e-2, 1.5]},
    "rho": {"x0": -0.7, "lbub": [-1, 1]},
    "lambd": {"x0": 0.4, "lbub": [-1, 1]}
}

inverseMaturitiesSum = (1/tau).sum()

x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]

In [134]:
def sqErr (x):
    v0, kappa, theta, sigma, rho, lambd = [param for param in x]
    
    # weight factor using inverse weighted average for maturities
    err = np.sum(1/tau/inverseMaturitiesSum*(price - heston_call_price(S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd))**2)
    # Penalty term: distance to initial parameter vector
    pen = np.mean(np.sqrt([(param-initial)**2 for param, initial in zip(x,x0)]))
    
    return err + pen


In [151]:
result = minimize(sqErr, x0, tol = 1e-3, method='SLSQP', options={'maxiter': 1e4 }, bounds=bnds)
result

/var/folders/ts/wphp397x1197fdj1s_5wjk040000gn/T/ipykernel_12599/3301616196.py:19: RuntimeWarning:

overflow encountered in exp

/var/folders/ts/wphp397x1197fdj1s_5wjk040000gn/T/ipykernel_12599/3301616196.py:21: RuntimeWarning:

invalid value encountered in multiply



 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.837173199378809
       x: [ 1.050e-01  2.000e+00  2.848e-01  5.023e-01  5.253e-02
            3.998e-01]
     nit: 12
     jac: [ 1.106e+00  1.498e-01 -3.856e-02 -2.197e-01 -1.501e-02
            1.791e-01]
    nfev: 96
    njev: 12

In [152]:
v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
v0, kappa, theta, sigma, rho, lambd

(np.float64(0.10497644019545932),
 np.float64(1.9999931939018631),
 np.float64(0.2848290354174377),
 np.float64(0.5022608311806471),
 np.float64(0.05253185511643074),
 np.float64(0.39975353135642794))

In [155]:
heston_prices = heston_call_price(S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd)
traded_options['HESTON_PRICE'] = heston_prices
traded_options

,UNDERLYING_LAST,QUOTE_DATE,MATURITIES,C_VOLUME,C_LAST,C_SIZE,C_BID,C_ASK,STRIKE,MARKET_PRICE,Date,RISK_FREE_RATE,HESTON_PRICE
4245,423.11,2023-11-01,0.024767,683.0,23.15,7 x 1,22.55,23.35,405.0,22.950,2023-11-01,0.0556,21.124386
4246,423.11,2023-11-01,0.024767,510.0,16.0,24 x 2,15.25,15.80,415.0,15.525,2023-11-01,0.0556,13.775950
4247,423.11,2023-11-01,0.024767,2024.0,12.95,4 x 6,12.35,12.75,420.0,12.550,2023-11-01,0.0556,10.722610
4248,423.11,2023-11-01,0.024767,629.0,10.15,7 x 10,10.10,10.20,425.0,10.150,2023-11-01,0.0556,8.121526
4249,423.11,2023-11-01,0.024767,1178.0,7.95,18 x 8,7.80,7.80,430.0,7.800,2023-11-01,0.0556,5.972397
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4543,467.70,2023-11-30,0.136986,1561.0,12.15,46 x 181,11.95,12.10,500.0,12.025,2023-11-30,0.0556,12.903975
4544,467.70,2023-11-30,0.136986,800.0,9.4,13 x 177,9.25,9.40,510.0,9.325,2023-11-30,0.0556,10.031799
4545,467.70,2023-11-30,0.136986,1186.0,3.95,111 x 202,4.00,4.15,540.0,4.075,2023-11-30,0.0556,4.147750
4546,467.70,2023-11-30,0.136986,1327.0,3.1,305 x 75,3.00,3.10,550.0,3.050,2023-11-30,0.0556,2.896825


In [156]:
# Visualization
fig = go.Figure(data = [go.Mesh3d(z=traded_options['MARKET_PRICE'], x=traded_options['MATURITIES'], y=traded_options['STRIKE'], colorscale='Viridis', opacity=0.55)])

# Add scatter plot
fig.add_trace(go.Scatter3d(z=traded_options['HESTON_PRICE'], x=traded_options['MATURITIES'], y=traded_options['STRIKE'], mode='markers', marker=dict(size=3, color='red', opacity=0.8)))

# Update layout
fig.update_layout(title='3D Surface Plot with Scatter Points', scene=dict(xaxis_title='Maturities', yaxis_title='Strike', zaxis_title='Price'))

# Show the figure
fig.show()

The results for parameters from optimization seems to be at a reasonable range, and the heston model price prediction overall matches decently well with the market price. It may be normal that the model doesn't fit certain points at the lower and upper end of strike prices due to low liquidity of these options, and the risk of overfitting the model. 

Still, there are many features that can be improved/ added in the future:
- Apply Jump diffusion to Heston model implementation for better calibration accuracy and speed
- Time dependant parameters if considering maturities with large time intervals
- C++ for speed
- Better calibration method for the dataset


# Model Validation using Monte Carlo methods

In [167]:
def heston_monte_carlo_prices(S0, v0, tau, r, K, sigma, rho, kappa, theta, paths, steps):
    option_values = []
    # Loop through each option (perform a monte carlo simulation for each option)
    for S, r, K, tau in zip(S0, r, K, tau):
        
        dt = tau / steps
        # Arrays to store the final asset prices
        final_prices = np.zeros(paths)

        # Loop for simulating paths 
        for i in range(paths):
            S_t = S
            v_t = v0
            for j in range(steps):
                # Generate correlated wiener processes for price and volatility
                Z1 = np.random.normal(0, 1)
                Z2 = rho * Z1 + np.sqrt(1 - rho**2) * np.random.normal(0, 1)
                
                # Applying heston discrete SDE 

                S_t = S_t*(np.exp((r- 0.5*v_t) * dt + np.sqrt(v_t) * Z1 * np.sqrt(dt)))
                v_t = np.abs(v_t + kappa * (theta - v_t) * dt + sigma * np.sqrt(v_t) * Z2 * np.sqrt(dt))
                
            final_prices[i] = S_t
            
        # Calculating option value by discounting expected payoff to present
        option_value = np.exp(-r * tau) * np.mean(np.maximum(final_prices - K, 0))
        option_values.append(option_value)
            
    return option_values

In [168]:
option_values = heston_monte_carlo_prices(S0, v0, tau, r, K, sigma, rho, kappa, theta, 100, 252)

In [169]:
traded_options['MONTE_CARLO_PRICE'] = option_values
traded_options

,UNDERLYING_LAST,QUOTE_DATE,MATURITIES,C_VOLUME,C_LAST,C_SIZE,C_BID,C_ASK,STRIKE,MARKET_PRICE,Date,RISK_FREE_RATE,HESTON_PRICE,MONTE_CARLO_PRICE
4245,423.11,2023-11-01,0.024767,683.0,23.15,7 x 1,22.55,23.35,405.0,22.950,2023-11-01,0.0556,21.124386,20.013043
4246,423.11,2023-11-01,0.024767,510.0,16.0,24 x 2,15.25,15.80,415.0,15.525,2023-11-01,0.0556,13.775950,12.602847
4247,423.11,2023-11-01,0.024767,2024.0,12.95,4 x 6,12.35,12.75,420.0,12.550,2023-11-01,0.0556,10.722610,10.106803
4248,423.11,2023-11-01,0.024767,629.0,10.15,7 x 10,10.10,10.20,425.0,10.150,2023-11-01,0.0556,8.121526,10.398564
4249,423.11,2023-11-01,0.024767,1178.0,7.95,18 x 8,7.80,7.80,430.0,7.800,2023-11-01,0.0556,5.972397,8.255680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4543,467.70,2023-11-30,0.136986,1561.0,12.15,46 x 181,11.95,12.10,500.0,12.025,2023-11-30,0.0556,12.903975,8.732651
4544,467.70,2023-11-30,0.136986,800.0,9.4,13 x 177,9.25,9.40,510.0,9.325,2023-11-30,0.0556,10.031799,10.762381
4545,467.70,2023-11-30,0.136986,1186.0,3.95,111 x 202,4.00,4.15,540.0,4.075,2023-11-30,0.0556,4.147750,6.052263
4546,467.70,2023-11-30,0.136986,1327.0,3.1,305 x 75,3.00,3.10,550.0,3.050,2023-11-30,0.0556,2.896825,8.171230


From the monte carlo simulation using the calibrated parameters, the results are close to the market price, which suggests that our calibration is done correctly and is at a good quality. Next step would be testing its performance (forward testing)

# Heston Model performance measure
Testing accuracy of calibrated parameters by comparing Black-Sholes Model and Heston Model prediction results for market price of options using performance metric RMSE.

In [170]:
# Using code above to clean and process options data
options_df_dec = pd.read_csv('/Users/wongmarco/Downloads/nvda_eod_2023q4-jf5cdq/nvda_eod_202312.txt')
options_df_dec.columns = options_df_dec.columns.str.replace('[\[\] ]', '', regex=True)
options_df_dec = options_df_dec.replace(' ',np.nan )
options_df_dec[['C_VOLUME','C_ASK','C_BID']] = options_df_dec[['C_VOLUME','C_ASK','C_BID']].astype(float)
options_df_dec.dropna()
options_df_dec = options_df_dec[options_df_dec['DTE'] != 0]
options_df_dec = options_df_dec[(options_df_dec['C_LAST'] != 0) & (options_df_dec['P_LAST'] != 0)]

traded_options_dec = options_df_dec[(options_df_dec['C_VOLUME'] > 500)&(options_df_dec['DTE']>7)&(options_df_dec['DTE']<366)][['UNDERLYING_LAST','DTE','C_BID','C_ASK','STRIKE']]
traded_options_dec['DTE'] = traded_options_dec['DTE']/365
traded_options_dec = traded_options_dec.rename(columns={'DTE': 'MATURITIES'})
traded_options_dec['MARKET_PRICE'] = (traded_options_dec['C_ASK']+traded_options_dec['C_BID'])/2
traded_options_dec = traded_options_dec[(traded_options_dec['MARKET_PRICE'] < 100) & (traded_options_dec['MARKET_PRICE'] > 1)]
traded_options_dec['RISK_NEUTRAL_RATE'] = 0.05

In [171]:
from scipy.stats import norm
S0, K, r, tau, price = traded_options_dec[['UNDERLYING_LAST', 'STRIKE','RISK_NEUTRAL_RATE', 'MATURITIES','MARKET_PRICE']].to_numpy().T

def black_scholes(S, K, T, r, sigma):
    
    # Calculate d1 and d2
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    price= S * norm.cdf(d1) - K * np.exp(-r*T)* norm.cdf(d2)
    
    return price

In [ ]:
# Optimize to find best parameters
result = minimize(objective_function, initial_params, args=(market_data,))

r

In [176]:
traded_options_dec['HESTON_PRICE'] = heston_call_price(S0, K, v0, tau, r, sigma, rho, kappa, theta, lambd) 
traded_options_dec['BLACK-SHOLES_PRICE'] = black_scholes(S0,K,tau,r,sigma)
traded_options_dec

,UNDERLYING_LAST,MATURITIES,C_BID,C_ASK,STRIKE,MARKET_PRICE,RISK_NEUTRAL_RATE,HESTON_PRICE,BLACK-SHOLES_PRICE
298,467.65,0.038356,12.35,12.50,467.5,12.425,0.05,12.680866,18.851461
299,467.65,0.038356,11.15,11.30,470.0,11.225,0.05,11.446398,17.657139
301,467.65,0.038356,8.95,9.00,475.0,8.975,0.05,9.222143,15.428827
303,467.65,0.038356,7.05,7.15,480.0,7.100,0.05,7.314054,13.409644
307,467.65,0.038356,4.25,4.35,490.0,4.300,0.05,4.366006,9.967203
...,...,...,...,...,...,...,...,...,...
50141,497.05,0.057534,9.05,9.25,510.0,9.150,0.05,10.970906,18.840458
50143,497.05,0.057534,6.20,6.35,520.0,6.275,0.05,7.594224,15.145402
50145,497.05,0.057534,4.00,4.15,530.0,4.075,0.05,5.040853,12.033086
50149,497.05,0.057534,1.74,1.80,550.0,1.770,0.05,1.859337,7.337417


In [177]:
print(np.sqrt(np.mean((traded_options_dec['MARKET_PRICE'] - traded_options_dec['HESTON_PRICE'])**2)))
print(np.sqrt(np.mean((traded_options_dec['MARKET_PRICE'] - traded_options_dec['BLACK-SHOLES_PRICE'])**2)))

0.9935481614670812
6.743180642065601
